In [12]:
import os
from google.colab import drive
drive.mount('/content/drive')

workshop_dir = '/content/drive/MyDrive/2022S_MODELING_WORKSHOP/'
if not os.path.isdir(workshop_dir):
  os.mkdir(workshop_dir)

os.chdir(workshop_dir)
if not os.path.isdir(workshop_dir + 'modelingworkshop'):
  !git clone https://github.com/CCBatIIT/modelingworkshop

lab_dir = workshop_dir + '2022S_MM_SESSION_3_1' #SPECIFIC TO EACH LAB
if not os.path.isdir(lab_dir):
  os.mkdir(lab_dir)
os.chdir(lab_dir)

# CONTENT SPECIFIC TO LAB
import json
try:
  import py3Dmol
except:
  !pip install py3Dmol
  import py3Dmol

mpro_pdb = '../modelingworkshop/data/bestmodel_aligned.pdb'
os.path.isfile(mpro_pdb), os.path.isdir('./pdb2amber/build_system')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


(True, True)

###STEP X - Protonating a protein

In [ ]:
!scp {mpro_pdb} ./mpro.pdb

view = py3Dmol.view()
view.addModel(open('mpro.pdb', 'r').read(),'pdb')
view.addStyle('stick')
view.setBackgroundColor('black')
view.zoomTo()
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
!pip install pdb2pqr -q
!pdb2pqr30 --help | head -n15

usage: pdb2pqr [-h] [--ff {AMBER,CHARMM,PARSE,TYL06,PEOEPB,SWANSON}]
               [--userff USERFF] [--clean] [--nodebump] [--noopt]
               [--keep-chain] [--assign-only]
               [--ffout {AMBER,CHARMM,PARSE,TYL06,PEOEPB,SWANSON}]
               [--usernames USERNAMES] [--apbs-input APBS_INPUT]
               [--pdb-output PDB_OUTPUT] [--ligand LIGAND] [--whitespace]
               [--neutraln] [--neutralc] [--drop-water] [--include-header]
               [--titration-state-method {propka}] [--with-ph PH]
               [-f FILENAMES] [-r REFERENCE] [-c CHAINS] [-i TITRATE_ONLY]
               [-t THERMOPHILES] [-a ALIGNMENT] [-m MUTATIONS] [-p PARAMETERS]
               [--log-level {DEBUG,INFO,WARNING,ERROR,CRITICAL}] [-o PH]
               [-w WINDOW WINDOW WINDOW] [-g GRID GRID GRID]
               [--mutator MUTATOR] [--mutator-option MUTATOR_OPTIONS] [-d]
               [-l] [-k] [-q] [--protonate-all] [--version]
               input_path output_pqr


In [ ]:
apbs_input = 'mpro_apbs'
out_pdb = 'mpro_H.pdb'
in_pdb = 'mpro.pdb'
out_pqr = 'mpro.pqr'
!pdb2pqr30 --ff=AMBER --nodebump --keep-chain --ffout=AMBER --apbs-input={apbs_input} --with-ph=7 --log-level=INFO --pdb-output={out_pdb} {in_pdb} {out_pqr}

INFO:PDB2PQR v3.5.2: biomolecular structure conversion software.
INFO:Please cite:  Jurrus E, et al.  Improvements to the APBS biomolecular solvation software suite.  Protein Sci 27 112-128 (2018).
INFO:Please cite:  Dolinsky TJ, et al.  PDB2PQR: expanding and upgrading automated preparation of biomolecular structures for molecular simulations. Nucleic Acids Res 35 W522-W525 (2007).
INFO:Checking and transforming input arguments.
INFO:Loading topology files.
INFO:Loading molecule: mpro.pdb
ERROR:Error parsing line: invalid literal for int() with base 10: 'Ato'
ERROR:<REMARK AtomGroup bestmodel>
ERROR:Truncating remaining errors for record type:REMARK

ERROR:['REMARK']
INFO:Setting up molecule.
INFO:Created biomolecule object with 612 residues and 4752 atoms.
INFO:Setting termini states for biomolecule chains.
INFO:Loading forcefield.
INFO:Loading hydrogen topology definitions.
INFO:This biomolecule is clean.  No repair needed.
INFO:Updating disulfide bridges.
INFO:Adding hydrogens to b

In [ ]:
view = py3Dmol.view()
view.addModel(open('mpro_H.pdb', 'r').read(),'pdb')
view.setBackgroundColor('black')
view.zoomTo()
view.show() #THIS SHOWS NO HS BUT I CHECKED THE PDB AND IT HAS THEM

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

##Heading

## STEP X - Paramterize the structure with AMBER force field

In [13]:
if not os.path.isdir('./pdb2amber'):
  !git clone https://github.com/swillow/pdb2amber

### Step X.1 - Parameterize the protein

In [24]:
sample_json = json.load(open('./pdb2amber/input.json', 'r'))
for key in sample_json:
  print(f'{key}: {sample_json[key]}')

fname_pdb: receptor.pdb
fname_prmtop: receptor.prmtop
fname_ff: ['./data/protein.ff14SB.xml', './data/wat_opc3.xml']


fname_pdb: receptor.pdb
fname_prmtop: receptor.prmtop
fname_ff: ['./data/protein.ff14SB.xml', './data/wat_opc3.xml', 'string', 'string']


###Step X.2 - Build Explicit Water Box

### Step X.3 - Parameterize the Protein and Explicit Water for a Neutral System